In [1]:
import pandas as pd
import numpy as np
from time import sleep
import missingno as msno
import matplotlib.pyplot as plt
import plotly.express as px
import geopandas as gpd
import fsspec
from shapely.geometry import Point
import seaborn as sns
pd.__version__

'2.3.3'

In [ ]:
# # Imports

# from eda_data_cleaning import DataCleaning
# from eda_hypothesis import Hypothesis3 , Hypothesis

# df = Hypothesis().df.copy()
# df.head()


# obj3 = Hypothesis3()


In [ ]:
# df.eval('price_per_sqft = price / sqft_living', inplace=True)
# df.head()
# # cols = df.columns
# df.sort_values(by='date')
# df.hist(figsize=(20,20), bins=5)

In [ ]:
# cols = np.array(df.columns)
# cols

In [ ]:
# df.plot(kind='scatter',x='date', y='price')
# df.plot(kind='scatter',x='price_per_sqft', y='zipcode')



In [ ]:
# mode = df.price_per_sqft.mode()
# max =df.price_per_sqft.max()
# min = df.price_per_sqft.min()
# mean = df.price_per_sqft.mean()

# print(min,max,mean)

In [ ]:
# # For apartment to be expensive they should be above the mean prices
# mode_price = df.price.mode()
# max_price =df.price.max()
# min_price = df.price.min()
# mean_price = df.price.mean()



# new = df[(df['price_per_sqft'] > mean) & (df['price'] > mean_price) & (df['sqft_living'] > 2500.55) & (df['waterfront'] == 0)] 
# new.hist(figsize=(20,20), bins=5)



In [ ]:
# fig = px.histogram(new, x='price_per_sqft', y='price')
# fig.show()

# Hypothesis - 3

Proofs:<br>
Location-Price Correlation: Show distance_from_center has stronger correlation with price than grade or condition

Central vs Peripheral Premium: Prove central areas command higher prices regardless of quality

Quality Independence: Show that in central areas, even low-grade properties are expensive, and in peripheral areas, even high-grade properties are relatively cheap

Visual Evidence: The geographic map should show clear price clustering by location, not scattered randomly by quality

In [ ]:
from eda_hypothesis import Hypothesis3

obj = Hypothesis3()
df = obj.prep_df()


In [ ]:
obj.vis_price_distr_cent(df)

In [ ]:
obj.vis_loc_vs_qlty_scatter(df)


In [ ]:
obj.vis_prc_by_grade_condition(df)

In [ ]:
# Geographic Evidence
# Zip code analysis
zipcode_stats = df.groupby('zipcode').agg({
    'price_per_sqft': 'mean',
    'lat': 'mean', 
    'long': 'mean',
    'grade': 'mean',
    'condition': 'mean'
}).sort_values('price_per_sqft', ascending=False)

# Premium Quantification
# Calculate exact premium
central_avg = df[df['is_central']]['price_per_sqft'].mean()
peripheral_avg = df[~df['is_central']]['price_per_sqft'].mean()
premium_pct = ((central_avg - peripheral_avg) / peripheral_avg) * 100

print(f"Central area premium: {premium_pct:.1f}%")


In [ ]:
import folium

m = folium.Map(location=[df['lat'].mean(), df['long'].mean()], zoom_start=11)
for idx, row in df.sample(1000).iterrows():  # Sample for clarity
    color = 'red' if row['price_per_sqft'] > df['price_per_sqft'].quantile(0.75) else 'blue'
    folium.CircleMarker(
        location=[row['lat'], row['long']],
        radius=3,
        popup=f"${row['price_per_sqft']:.0f}/sqft<br>Grade: {row['grade']}",
        color=color,
        fill=True
    ).add_to(m)

# m.show_in_browser()

In [ ]:
corr_matrix = df[['price_per_sqft', 'distance_from_center', 'grade', 'condition', 'sqft_living']].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation: Location vs Other Factors')


# # Compute correlation matrix
# corr_matrix = df[['price_per_sqft', 'distance_from_center', 'grade', 'condition', 'sqft_living']].corr()
# # Create a mask for the upper triangle
# mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
# # Set up the matplotlib figure
# plt.figure(figsize=(8, 6))
# # Draw the heatmap with the mask
# sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm', center=0)
# plt.title('Correlation: Location vs Other Factors')
# plt.tight_layout()
# plt.show()

In [ ]:
central_low_quality = df[(df['is_central']) & (df['grade'] <= 7)]
peripheral_high_quality = df[(~df['is_central']) & (df['grade'] >= 10)]

# Expected: central_low_quality.mean_price > peripheral_high_quality.mean_price

# Visualization: Side-by-side box plots
fig, ax = plt.subplots(1, 2, figsize=(15, 6))
sns.boxplot(data=central_low_quality, x='grade', y='price_per_sqft', ax=ax[0])
sns.boxplot(data=peripheral_high_quality, x='grade', y='price_per_sqft', ax=ax[1])

# Find specific examples that prove the point
examples = pd.DataFrame({
    'Type': ['Central_LowGrade', 'Peripheral_HighGrade'],
    'Avg_Price_SqFt': [
        central_low_quality['price_per_sqft'].mean(),
        peripheral_high_quality['price_per_sqft'].mean()
    ],
    'Avg_Grade': [
        central_low_quality['grade'].mean(),
        peripheral_high_quality['grade'].mean()
    ]
})

print("Compelling Evidence:")
print(examples)

# Hypothesis 2

In [ ]:
from eda_hypothesis import Hypothesis2, Hypothesis3

obj2 = Hypothesis2()
df = obj2.prep_df()


In [ ]:
obj2.vis_qlty_roi(df)


In [ ]:
obj2.vis_dimnish_returns(df)


In [ ]:
obj2.vis_interaction_effects(df)

In [ ]:
obj2.vis_main_plot(df)


# Hypothesis 1

In [ ]:
from eda_hypothesis import Hypothesis, FinalRecommendation

In [ ]:
df = Hypothesis().df_2.copy()
Hypothesis().run_complete_hypothesis_1_analysis(df)

# Final Recommendation

In [ ]:
from eda_hypothesis import Hypothesis, FinalRecommendation

map, top_3 = FinalRecommendation().run_property_recommendation_with_map(df)